# Notebook: Linting and Static Analysis Test Suite

This notebook executes targeted tests for Python linting, formatting, static type checking, and complexity analysis tools. Specifically, it validates integration and functionality of:

- **Black** (auto-formatting)
- **Ruff** (linting and style checking)
- **Mypy** (static type checking)
- **Radon** (complexity analysis)

These tests confirm the tools' abilities to detect, report, and correct common Python code quality issues. Within the overall system, this notebook supports automated quality control processes, ensuring consistent code readability, correctness, and maintainability for both human developers and AI agents.


### Configuring Project Path

This cell ensures the notebook can locate the project's modules by adding the project root directory to Python’s import path (`sys.path`). This setup is required for relative imports from the main application (`app`) to work correctly within the notebook environment.


In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent.parent 
sys.path.insert(0, str(PROJECT_ROOT))


### Importing Dependencies and Tool Providers

This cell imports all necessary libraries, including `black`, `ruff`, `mypy`, and `radon`, and initializes logging. It also registers custom tool providers from the application's extensions to ensure tests execute correctly.


In [4]:
import logging
from importlib import import_module
from pathlib import Path
import subprocess
import black
import ruff
import mypy
import radon

from app.factories.tool_provider import ToolProviderFactory

import_module("app.extensions.tool_providers")
logging.basicConfig(level=logging.DEBUG)


### Defining Code Snippets for Linting and Analysis

This cell defines Python code snippets used in subsequent linting and static analysis tests. The examples include both valid code and intentionally problematic code to trigger specific tool behaviors:

- **`good_code`**: Correct, well-formatted Python function; baseline for all tests.
- **`bad_linting`**: Poor formatting intended to trigger formatting corrections.
- **`bad_typing`**: Intentional type mismatch to test type-checking tools.
- **`complex_code`**: Nested loops used to trigger complexity warnings.
- **`missing_import`**: Unused import statement to trigger linting errors.
- **`invalid_code`**: Invalid Python syntax to test parsing and error detection.


In [6]:
# Correct example
good_code = """def add(x: int, y: int) -> int:
    return x + y
"""

# Linting errors example: bad formatting, spacing issues
bad_linting = """x=1


def foo():
    return 1
"""

# Typing error example: function declares int return but returns str
bad_typing = """def add(x: int, y: int) -> int:
    return str(x + y)
"""

# Complexity issue example (nested loops)
complex_code = """def complex(n):
    for i in range(n):
        if i % 2 == 0:
            for j in range(n):
                if j % 3 == 0:
                    print(i, j)
"""

# Missing import example (unused import)
missing_import = """import os


def bar():
    return 42
"""

# Invalid Python syntax example
invalid_code = """def oops(:
    pass
"""


### Testing Black Formatter

**Black** is a Python formatter that enforces consistent code styling automatically. It corrects spacing, indentation, line length, and formatting issues.

This test demonstrates how **Black** transforms code snippets:

- **`good_code`**: Already well-formatted; minimal or no changes expected.
- **`bad_linting`**: Poorly formatted; demonstrates how Black fixes common style issues.

The output shows the formatted code and any relevant messages from Black, verifying that code style consistency is maintained.


In [7]:
def run_black(code):
    path = Path("black_demo.py")
    path.write_text(code)
    provider = ToolProviderFactory.create("black")
    try:
        result = provider.run(str(path))
        print(result.stdout)
        print(result.stderr)
        print(path.read_text())
    finally:
        path.unlink()

print('Black on good code:')
run_black(good_code)
print('Black on linting issues:')
run_black(bad_linting)


DEBUG:BlackToolProvider:Tool run start


Black on good code:


DEBUG:BlackToolProvider:Tool run end
DEBUG:BlackToolProvider:Tool run start




def add(x: int, y: int) -> int:
    return x + y

Black on linting issues:


DEBUG:BlackToolProvider:Tool run end




x = 1


def foo():
    return 1



### Testing Ruff Linter

**Ruff** detects common Python code-quality issues such as unused imports, syntax errors, and formatting problems.

This test demonstrates Ruff’s capabilities:

- **`good_code`**: Clean; no errors expected.
- **`missing_import`**: Ruff correctly identifies and reports an unused import (`os`), demonstrating its lint detection.
- **`invalid_code`**: Invalid syntax; Ruff will detect and report syntax errors clearly.

The output includes detected errors, their exact locations, and whether Ruff can automatically fix the issues (e.g., unused imports via the `--fix` option). Return codes (`0` for success, non-zero for detected issues) confirm linting results clearly.


In [9]:
def run_ruff(code):
    path = Path("ruff_demo.py")
    path.write_text(code)
    provider = ToolProviderFactory.create("ruff")
    try:
        result = provider.run(str(path))
        print(result.stdout)
        print(result.stderr)
        print('return code', result.returncode)
    except Exception as e:
        print(f'Error running Ruff: {e}')
    finally:
        path.unlink()

print('Ruff on good code:')
run_ruff(good_code)
print('Ruff with unused import:')
run_ruff(missing_import)
print('Ruff on invalid code:')
run_ruff(invalid_code)


DEBUG:RuffToolProvider:Tool run start
DEBUG:RuffToolProvider:All checks passed!

DEBUG:RuffToolProvider:Tool run end
DEBUG:RuffToolProvider:Tool run start


Ruff on good code:
All checks passed!


return code 0
Ruff with unused import:


DEBUG:RuffToolProvider:ruff_demo.py:1:8: F401 [*] `os` imported but unused
Found 1 error.
[*] 1 fixable with the `--fix` option.

DEBUG:RuffToolProvider:Tool run start
DEBUG:RuffToolProvider:ruff_demo.py:1:10: E999 SyntaxError: Expected a parameter or the end of the parameter list
Found 1 error.

ERROR:RuffToolProvider:error: Failed to parse ruff_demo.py:1:10: Expected a parameter or the end of the parameter list



Error running Ruff: ruff failed: 
Ruff on invalid code:
Error running Ruff: ruff failed: error: Failed to parse ruff_demo.py:1:10: Expected a parameter or the end of the parameter list



### Testing Mypy Static Type Checker

**Mypy** is a static type checker that verifies Python code against type annotations, catching type-related errors before runtime.

This test demonstrates:

- **`good_code`**: Correct annotations; expected to pass without issues.
- **`bad_typing`**: Contains a deliberate type mismatch; Mypy detects and clearly reports the incompatible return type.
- **`invalid_code`**: Contains syntax errors; Mypy immediately reports these parsing issues.

The output clearly shows detected type and syntax errors, their locations, error messages, and Mypy’s return codes (`0` for success, non-zero for issues). Exception handling ensures all tests run to completion.


In [11]:
def run_mypy(code):
    path = Path("mypy_demo.py")
    path.write_text(code)
    provider = ToolProviderFactory.create("mypy")
    try:
        result = provider.run(str(path))
        print(result.stdout)
        print(result.stderr)
        print('return code', result.returncode)
    except Exception as e:
        print(f'Error running Mypy: {e}')
    finally:
        path.unlink()

print('Mypy on good code:')
run_mypy(good_code)

print('Mypy with typing error:')
run_mypy(bad_typing)

print('Mypy on invalid code:')
run_mypy(invalid_code)


DEBUG:MypyToolProvider:Tool run start


Mypy on good code:


DEBUG:MypyToolProvider:Success: no issues found in 1 source file

DEBUG:MypyToolProvider:Tool run end
DEBUG:MypyToolProvider:Tool run start


Success: no issues found in 1 source file


return code 0
Mypy with typing error:


DEBUG:MypyToolProvider:mypy_demo.py:2: error: Incompatible return value type (got "str", expected "int")  [return-value]
Found 1 error in 1 file (checked 1 source file)

DEBUG:MypyToolProvider:Tool run end
DEBUG:MypyToolProvider:Tool run start
DEBUG:MypyToolProvider:mypy_demo.py:1: error: invalid syntax  [syntax]
Found 1 error in 1 file (errors prevented further checking)



mypy_demo.py:2: error: Incompatible return value type (got "str", expected "int")  [return-value]
Found 1 error in 1 file (checked 1 source file)


return code 1
Mypy on invalid code:
Error running Mypy: mypy execution error: 


### Testing Radon Complexity Analyzer

**Radon** analyzes Python code to measure its cyclomatic complexity, which helps identify overly complex functions that may be hard to maintain or test.

This test demonstrates:

- **`good_code`**: Simple, low-complexity function; expected complexity rating is 'A'.
- **`complex_code`**: Function with nested loops intended to increase complexity; complexity rating reflects higher complexity.

The output shows Radon’s complexity ratings clearly for each tested snippet. Ratings range from 'A' (low complexity) to 'F' (high complexity).


In [13]:
def run_radon(code):
    path = Path("radon_demo.py")
    path.write_text(code)
    provider = ToolProviderFactory.create("radon")
    try:
        result = provider.run(str(path))
        print(result.stdout)
    except Exception as e:
        print('Radon failed:', e)
    finally:
        path.unlink()

print('Radon on good code:')
run_radon(good_code)
print('Radon on complex code:')
run_radon(complex_code)


DEBUG:RadonToolProvider:Tool run start


Radon on good code:


DEBUG:RadonToolProvider:radon_demo.py
    F 1:0 add - A

DEBUG:RadonToolProvider:Tool run end
DEBUG:RadonToolProvider:Tool run start


radon_demo.py
    F 1:0 add - A

Radon on complex code:


DEBUG:RadonToolProvider:radon_demo.py
    F 1:0 complex - A

DEBUG:RadonToolProvider:Tool run end


radon_demo.py
    F 1:0 complex - A

